In [1]:
import pandas as pd
import sys
import os

In [18]:
path = "/Users/irbraun/phenologs-with-oats/outputs/stacked_10_12_2020_h08m36s56_4218_plant"


results = pd.read_csv(os.path.join(path, "stacked_full_table_with_all_metrics.csv"))
results

,method,species,objective,curated,hyperparameters,group,order,mean_1,mean_0,n_1,n_0,ks,ks_pval,auc,auc_baseline,f1_max_baseline,f1_max,f5_max,f2_max
0,baseline,intra,subsets,True,identity,nlp,0,0.999834,0.999992,708849.0,1025348.0,0.000159,1.000000e+00,0.672677,0.408748,0.580299,0.580299,0.463564,0.775615
1,baseline,both,pathways,False,identity,nlp,0,0.999151,0.997767,380647.0,1038558.0,0.001384,6.592867e-01,0.195544,0.268211,0.422976,0.422976,0.314197,0.646964
2,baseline,inter,pathways,False,identity,nlp,0,1.000000,1.000000,7463.0,261043.0,0.000000,1.000000e+00,0.513897,0.027795,0.054086,0.054086,0.034503,0.125068
3,baseline,intra,pathways,False,identity,nlp,0,0.999134,0.997017,373184.0,777515.0,0.002117,2.081115e-01,0.223576,0.324311,0.489780,0.489780,0.374985,0.705869
4,baseline,intra,predicted,False,identity,nlp,0,0.998731,0.993531,293881.0,5374621.0,0.005200,5.705286e-07,0.031831,0.051845,0.098578,0.098578,0.063976,0.214699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,biobert,intra,predicted,True,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.009225,0.048261,252435.0,1458171.0,0.595606,0.000000e+00,0.610469,0.147571,0.257188,0.513391,0.421490,0.673779
794,biobert,intra,pathways,False,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.007582,0.062912,373184.0,777515.0,0.790208,0.000000e+00,0.884371,0.324311,0.489780,0.849183,0.848507,0.878046
795,biobert,both,pathways,True,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.004719,0.055322,345309.0,307525.0,0.783811,0.000000e+00,0.932893,0.528938,0.691903,0.901536,0.896629,0.924791
796,biobert,intra,subsets,False,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.011904,0.064557,714581.0,1096278.0,0.765122,0.000000e+00,0.929594,0.394609,0.565906,0.862204,0.910280,0.864675


In [19]:
results.dtypes

method              object
species             object
objective           object
curated               bool
hyperparameters     object
group               object
order                int64
mean_1             float64
mean_0             float64
n_1                float64
n_0                float64
ks                 float64
ks_pval            float64
auc                float64
auc_baseline       float64
f1_max_baseline    float64
f1_max             float64
f5_max             float64
f2_max             float64
dtype: object

In [21]:
variable_combinations = [
    (True,"subsets","intra"),
    (True,"known","intra"),
    (True,"predicted","intra"),
    (True,"pathways","intra"),
    (True,"pathways","inter"),
    (True,"pathways","both"),
    (True,"orthologs","inter"),
    (False,"subsets","intra"),
    (False,"known","intra"),
    (False,"predicted","intra"),
    (False,"pathways","intra"),
    (False,"pathways","inter"),
    (False,"pathways","both"),
    (False,"orthologs","inter"),
]


In [60]:
# Make another version of the table that is more useful for looking at one particular metric or value.
metrics_of_interest = ["f5_max", "f2_max"]
for metric_of_interest in metrics_of_interest:
    reshaped_results = results[["method","hyperparameters","order","group"]].drop_duplicates()
    for (c,q,s) in variable_combinations:

        # Construct a column name that corresponds to a particular subset.
        c_str = {True:"curated",False:"all"}[c]
        q_str = str(q).lower()
        s_str = str(s).lower()
        col_name =  "{}_{}_{}".format(s_str, c_str, q_str)

        # Pull data out of the the full metrics dataframe.
        reshaped_results[col_name] = reshaped_results["order"].map(lambda x: results.loc[(results["order"]==x) & (results["curated"]==c) & (results["objective"]==q.lower()) & (results["species"]==s.lower()), metric_of_interest])
        reshaped_results[col_name] = reshaped_results[col_name].map(lambda x: None if len(x)==0 else x.values[0])
    
    # Remove columns that have all NA values, these are for questions that weren't applicable to these metrics.
    #reshaped_results.dropna(axis="columns", how="all", inplace=True)
    reshaped_results.to_csv(os.path.join(path, "stacked_{}.csv").format(metric_of_interest), index=False)
